In [ ]:
# !pip install keras

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install pillow

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install pydot-ng

In [ ]:
!pip install pydotplus

In [ ]:
!pip install graphviz

In [8]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from keras.layers import Activation, Input
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

In [9]:
from keras.layers import Input, Activation, Dense, Conv1D, Dropout, BatchNormalization, Reshape

In [10]:
from keras.layers import Add, Lambda, Concatenate, SpatialDropout1D

import keras
from keras.layers import Input, Activation, Dense, Conv1D, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.models import load_model, Model
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU
from  keras import backend as K

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback

In [12]:
from keras.models import load_model

In [34]:
from keras.utils import plot_model

In [13]:
from PIL import Image

In [ ]:
batch_size = 64
target_size = (256,256)
color_mode = 'grayscale'
dataset = '/raid/audio_detection/audio_detection/data'
dataset_1= 'new'
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
print(os.getcwd())
print(os.chdir(dataset))
print(os.getcwd())


train_generator = train_datagen.flow_from_directory(f'{dataset}/train/', target_size=target_size, batch_size = batch_size,class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
val_generator = val_datagen.flow_from_directory(f'{dataset}/val/', target_size=target_size, batch_size = batch_size,class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
test_generator = test_datagen.flow_from_directory(f'{dataset}/test/', target_size=target_size, batch_size = batch_size,class_mode='binary', color_mode=color_mode, classes = ['real','fake'])


In [15]:
x,y = next(train_generator)

In [ ]:
y

In [ ]:
x,y = next(test_generator)
x

In [18]:
def customPooling(x):
    target = x[1]
    inputs = x[0]
    maskVal = 0
    #getting the mask by observing the model's inputs
    mask = K.equal(inputs, maskVal)
    mask = K.all(mask, axis=-1, keepdims=True)

    #inverting the mask for getting the valid steps for each sample
    mask = 1 - K.cast(mask, K.floatx())

    #summing the valid steps for each sample
    stepsPerSample = K.sum(mask, axis=1, keepdims=False)

    #applying the mask to the target (to make sure you are summing zeros below)
    target = target * mask

    #calculating the mean of the steps (using our sum of valid steps as averager)
    means = K.sum(target, axis=1, keepdims=False) / stepsPerSample

    return means

In [19]:
def build_custom_convnet():
    #K.clear_session()
    input_shape  = (256,256,1)
    img_input = Input(input_shape, name='image_input')
    #Reshape((3, 4), input_shape=(12,))
    image_input = Reshape((256,256), input_shape= input_shape)(img_input)
    num_conv_blocks = 8
    init_neurons = 32
    spatial_dropout_fraction = 0.05
    num_dense_layers = 2
    num_dense_neurons = 50
    learning_rate = 0.0001
    #model_params['residual_con'] = 2
    convnet = []
    convnet_5 = []
    convnet_7 = []
    for ly in range(0, num_conv_blocks):
        if ly == 0:
            convnet.append(Conv1D(init_neurons, 3, strides=1, activation='linear', padding='causal')(image_input))
            print(convnet)
            convnet_5.append(Conv1D(init_neurons, 5, strides=1, activation='linear', padding='causal')(image_input))
            convnet_7.append(Conv1D(init_neurons, 7, strides=1, activation='linear', padding='causal')(image_input))
        else:
            convnet.append(
                Conv1D(init_neurons * (ly * 2), 3, strides=1, activation='linear', padding='causal')(convnet[ly - 1]))
            convnet_5.append(
                Conv1D(init_neurons * (ly * 2), 5, strides=1, activation='linear', padding='causal')(convnet_5[ly - 1]))
            convnet_7.append(
                Conv1D(init_neurons * (ly * 2), 7, strides=1, activation='linear', padding='causal')(convnet_7[ly - 1]))

        convnet[ly] = LeakyReLU()(convnet[ly])
        convnet_5[ly] = LeakyReLU()(convnet_5[ly])
        convnet_7[ly] = LeakyReLU()(convnet_7[ly])
        if (ly - 2) >= 0:
            res_conv = Conv1D(init_neurons * (ly * 2), 1, strides=1, activation='linear', padding='same')(
                convnet[ly - 2])
            convnet[ly] = Add(name=f'residual_con_3_{ly}')([convnet[ly], res_conv])
            res_conv_5 = Conv1D(init_neurons * (ly * 2), 1, strides=1, activation='linear', padding='same')(
                convnet_5[ly - 2])
            convnet_5[ly] = Add(name=f'residual_con_5_{ly}')([convnet_5[ly], res_conv_5])
            res_conv_7 = Conv1D(init_neurons * (ly * 2), 1, strides=1, activation='linear', padding='same')(
                convnet_7[ly - 2])
            convnet_7[ly] = Add(name=f'residual_con_7_{ly}')([convnet_7[ly], res_conv_7])

        if ly<(num_conv_blocks-1):
            convnet[ly] = SpatialDropout1D(spatial_dropout_fraction)(convnet[ly])
            convnet_5[ly] = SpatialDropout1D(spatial_dropout_fraction)(convnet_5[ly])
            convnet_7[ly] = SpatialDropout1D(spatial_dropout_fraction)(convnet_7[ly])

    dense = Lambda(lambda x: customPooling(x))([image_input,convnet[ly]])
    dense_5 = Lambda(lambda x: customPooling(x))([image_input,convnet_5[ly]])
    dense_7 = Lambda(lambda x: customPooling(x))([image_input,convnet_7[ly]])

    dense = Concatenate()([dense, dense_5, dense_7])

    for layers in range(num_dense_layers):
        dense = Dense(num_dense_neurons, activation='linear')(dense)
        dense = BatchNormalization()(dense)
        dense = LeakyReLU()(dense)
        dense = Dropout(0)(dense)
        print(0)
    output_layer = Dense(1)(dense)
    output_layer = Activation('sigmoid')(output_layer)
    model = Model(inputs=img_input, outputs=output_layer)
    opt = optimizers.Adam(lr=learning_rate)
    print("1")
    model.summary()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [30]:
model = load_model("/raid/audio_detection/audio_detection/data/models/raid/audio_detection/audio_detection/data_0tcn-janavi-epochs:20_acc:0.9801.h5")

In [31]:
import os     
os.environ["PATH"] += os.pathsep + '/usr/local/lib/python3.6/dist-packages'

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
predictions = model.predict(x)
predictions.shape


In [ ]:
pred = pd.DataFrame(predictions)      

In [ ]:
pred.describe()

In [ ]:
np.reshape(predictions,(64,))

In [ ]:
#predictions[:].astype(int)
for i in predictions:
    if i[0] < 0.5:
        i[0] = 0.0
    else :
        i[0] = 1.0

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
matrix = confusion_matrix(y,predictions, labels=[1,0])

In [ ]:
print(matrix)

In [ ]:
matrix = classification_report(y,predictions,labels=[1,0])
print('Classification report : \n',matrix)